# Source
http://www.valorinutritivi.ch/request?xml=MessageData&xml=MetaData&xsl=Download&lan=it&pageKey=Start

In [1]:
import pandas as pd
from pprint import pprint

In [2]:
path = "../raw/Swiss Food Comp Data V5.3.xlsx"

In [3]:
df = pd.read_excel(path)

In [4]:
df.columns = df.iloc[1]
df.reindex(df.index.drop(1))
df = df.drop([0,1])
df = df.drop(columns=['ID','ID V 4.0','ID SwissFIR','name D','synonyms D','name F','synonyms F','name E','synonyms E','category D','category F',
 'category E'])
df.columns = ["name","synonyms","category"] + list(df.columns[3:])
df = df.drop(columns=["record has changed",'matrix unit', 'value type', 'source'])
df = df.set_index("name")

In [5]:
df.describe()

,synonyms,category,specific gravity,energy kJ,unit,energy kcal,unit,protein,unit,alcohol,...,magnesium (Mg),unit,phosphorus (P),unit,iron (Fe),unit,iodide (I),unit,zinc (Zn),unit
count,96,999,124.0,999,999,999,999,999.0,999,996.0,...,991.0,991,991.0,991,992.0,992,970.0,970,992.0,992
unique,67,118,11.0,584,1,426,1,285.0,1,28.0,...,156.0,1,172.0,1,83.0,1,144.0,1,70.0,1
top,Fettina,Verdure/Verdure cotte (incl. conserve),1.0,1490,kilojoule,30,kilocalorie,0.0,gram,0.0,...,18.0,milligram,110.0,milligram,0.5,milligram,0.0,microgram,0.1,milligram
freq,4,53,60.0,12,999,13,999,36.0,999,951.0,...,46.0,991,40.0,991,72.0,992,71.0,970,91.0,992


In [6]:
# Drop energy in kj
df = df.iloc[:,[i for i,x in enumerate(df.columns) if i not in [3,4]]]

In [7]:
conversion = {
    "kilocalorie":1,
    "gram":1,
    "milligram":1e-3,
    "microgram":1e-6
}

In [8]:
# normalize the data
cols_to_skips = []
for i in range(5,len(df.columns),2):
    if "retinol equivalent" in df.iloc[:,i+1].tolist():
        cols_to_skips.append(i)
        continue 
    if "beta-carotene equivalent" in df.iloc[:,i+1].tolist():
        cols_to_skips.append(i)
        continue 
    if "alpha-tocopherol equivalent" in df.iloc[:,i+1].tolist():
        cols_to_skips.append(i)
        continue 
    df.iloc[:,i] *= [conversion[x] if not pd.isna(x) else 1 for x in df.iloc[:,i+1] ]

In [9]:
#drop unit cols
index = [0,1,2] + list(range(3,len(df.columns),2))
print(index)
for i in cols_to_skips:
    index.remove(i)
df = df.iloc[:,index]

[0, 1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73]


In [13]:
import re
regex = r"(\([^\)]+\))"
df.columns = [ re.sub(regex, "", x, 0, re.MULTILINE).lower().strip() for x in df.columns]

In [15]:
df.to_csv("../csv/confederazione_svizzera.csv")